In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.7 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


^C
^C


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import spacy
from tqdm import tqdm

# Install tqdm if not already installed
!pip install tqdm

# Enable tqdm for pandas
tqdm.pandas()

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

# Load Haunted Places dataset
df = pd.read_csv("./haunted_places_v2.tsv", sep="\t")

# Function to extract all named entities and organize them
def extract_entities(text):
    doc = nlp(str(text))
    all_entities = [(ent.text, ent.label_) for ent in doc.ents]
    persons = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
    orgs = [ent.text for ent in doc.ents if ent.label_ == "ORG"]
    locations = [ent.text for ent in doc.ents if ent.label_ in ("GPE", "LOC", "FAC")]
    return pd.Series([str(all_entities), str(persons), str(orgs), str(locations)])

# Apply extraction function with progress bar
df[["NER_ENTITIES", "NER_PERSONS", "NER_ORGS", "NER_LOCATIONS"]] = df["description"].progress_apply(extract_entities)

# Save the updated dataset as TSV
df.to_csv("haunted_places_v2.tsv", sep='	', index=False)

100%|██████████| 10992/10992 [03:41<00:00, 49.60it/s]


In [ ]:


# Function to compute percentage of non-empty entries in a given column
def percent_with_entities(column_name):
    return df[column_name].apply(lambda x: len(ast.literal_eval(x)) > 0).mean() * 100

# Compute percentage for each entity category
percent_entities = percent_with_entities("NER_ENTITIES")
percent_persons = percent_with_entities("NER_PERSONS")
percent_orgs = percent_with_entities("NER_ORGS")
percent_locations = percent_with_entities("NER_LOCATIONS")

# Print results
print(f"Percentage with at least one entity (any type):   {percent_entities:.2f}%")
print(f"Percentage with at least one PERSON entity:       {percent_persons:.2f}%")
print(f"Percentage with at least one ORG entity:          {percent_orgs:.2f}%")
print(f"Percentage with at least one LOCATION entity:     {percent_locations:.2f}%")

Percentage with at least one entity (any type):   83.43%
Percentage with at least one PERSON entity:       26.14%
Percentage with at least one ORG entity:          26.96%
Percentage with at least one LOCATION entity:     21.72%


In [ ]:
import pandas as pd
from collections import Counter
import ast

# Load the updated dataset
df = pd.read_csv("./haunted_places_v2.tsv", sep="\t")

# Convert stringified lists to Python lists
for col in ["NER_ENTITIES", "NER_PERSONS", "NER_ORGS", "NER_LOCATIONS"]:
    df[col] = df[col].apply(ast.literal_eval)

# Top Locations of Places
location_counter = Counter()
for locs in df["NER_LOCATIONS"]:
    location_counter.update(locs)

top_locations = location_counter.most_common(10)
print("Top 10 Mentioned Locations in Haunted Place Descriptions:")
for i, (loc, count) in enumerate(top_locations, 1):
    print(f"{i}. {loc}: {count} times")

# Looking at Cities matching Locations
if "city" in df.columns:
    df["city_match_in_locations"] = df.apply(
        lambda row: row["city"] in row["NER_LOCATIONS"] if pd.notnull(row["city"]) else False,
        axis=1
    )
    match_count = df["city_match_in_locations"].sum()
    total_rows = len(df)
    match_rate = match_count / total_rows

    print(f"Cities Found in Location Entities: {match_count}/{total_rows} rows")
    print(f"Proportion of rows where the city is mentioned in the extracted location entities: {match_rate:.2%}")
else:
    print("No 'city' column found to correlate with NER_LOCATIONS.")

# PERSON Entity Analysis – Contextual Insight
df["person_entity_count"] = df["NER_PERSONS"].apply(len)
avg_persons = df["person_entity_count"].mean()
percent_with_persons = (df["person_entity_count"] > 0).mean() * 100
max_persons = df["person_entity_count"].max()
num_with_multiple = (df["person_entity_count"] >= 3).sum()

print("\nAnalysis of PERSON Entities in Haunted Place Descriptions:")
print(f"- Average PERSON entities per description: {avg_persons:.2f}")
print(f"- % of entries with at least one PERSON: {percent_with_persons:.2f}%")
print(f"- Max PERSON entities in a single description: {max_persons}")
print(f"- Entries with 3 or more PERSON mentions: {num_with_multiple} ({num_with_multiple / len(df):.2%} of entries)")


Top 10 Mentioned Locations in Haunted Place Descriptions:
1. Washington: 22 times
2. Chicago: 21 times
3. US: 19 times
4. Mansion: 19 times
5. Ohio: 18 times
6. Main Street: 18 times
7. Auditorium: 17 times
8. Michigan: 16 times
9. East: 15 times
10. Texas: 14 times
Cities Found in Location Entities: 531/10992 rows
Proportion of rows where the city is mentioned in the extracted location entities: 4.83%

Analysis of PERSON Entities in Haunted Place Descriptions:
- Average PERSON entities per description: 0.42
- % of entries with at least one PERSON: 26.14%
- Max PERSON entities in a single description: 17
- Entries with 3 or more PERSON mentions: 375 (3.41% of entries)


In [ ]:
import ast

# Load both datasets
df_haunted = pd.read_csv("haunted_places_v2_tsv", sep="\t"')
df_captions = pd.read_csv("output_with_captions.tsv", sep="\t")

# Convert the NER_ENTITIES column
df_haunted["NER_ENTITIES"] = df_haunted["NER_ENTITIES"].apply(ast.literal_eval)

# Extract just the entity text from the NER_ENTITIES list
df_haunted["entity_texts"] = df_haunted["NER_ENTITIES"].apply(lambda ents: [ent[0] for ent in ents])

# Combine the two datasets for row-wise comparison
df_combined = pd.concat([df_haunted, df_captions], axis=1)

# Check if any entity text is found in the corresponding caption (case insensitive)
def has_entity_in_caption(entity_list, caption):
    caption_lower = str(caption).lower()
    return any(entity.lower() in caption_lower for entity in entity_list)

df_combined["entity_in_caption"] = df_combined.apply(
    lambda row: has_entity_in_caption(row["entity_texts"], row["caption"]),
    axis=1
)

# Count how many captions contain at least one identified entity
count_with_entity = df_combined["entity_in_caption"].sum()
total_rows = len(df_combined)
percentage = (count_with_entity / total_rows) * 100

print(f" {count_with_entity} out of {total_rows} captions ({percentage:.2f}%) contain at least one identified entity from the corresponding haunted place description.")



✅ 2378 out of 10992 captions (21.63%) contain at least one identified entity from the corresponding haunted place description.
